### Expansión de documentos
En este proyecto se ha utilizado el modelo DocTTTTTquery. Este modelo permite generar texto a partir de otro dado de diferentes formas. Las opciones que se han probado en el proyecto son: generación de resúmenes, generación de respuesta y generación de preguntas. Observando los resultados se ha concluido que la forma de expansión que más información aporta es la de generación de preguntas, ya que es la que más vocabulario nuevo incluye. De esta forma, para extender los documentos se han realizado preguntas que el texto pueda responder, y se han añadido al final de los textos. Esto aporta contenido nuevo, sin perder información ni perder el tema importante del documento. 

La expansión de documentos se ha realizado sobre le conjunto de entrenamiento, sin modificar ni expandir el conjunto de test.

In [10]:
import pandas as pd
df = pd.read_json("dataTrain.json")

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'doc2query/all-with_prefix-t5-base-v1'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
prefix = "answer2question"
outputColumn = []
cont = 0
for i in df['text']:
    text = prefix+": "+ i
    input_ids = tokenizer.encode(text, max_length=384, truncation=True, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=20)

    outputText = i

    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        outputText = outputText + ' ' +  query
    outputColumn.append(outputText)
    if cont % 100 == 0:
        print(cont)
        print(cont/df.size)
    cont += 1
df['expandedText'] = outputColumn

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0
0.0
100
0.005032965926820675
200
0.01006593185364135
300
0.015098897780462027
400
0.0201318637072827
500
0.025164829634103377
600
0.030197795560924053
700
0.035230761487744726
800
0.0402637274145654
900
0.04529669334138608
1000
0.050329659268206754
1100
0.05536262519502743
1200
0.060395591121848106
1300
0.06542855704866878
1400
0.07046152297548945
1500
0.07549448890231013
1600
0.0805274548291308
1700
0.08556042075595148
1800
0.09059338668277216
1900
0.09562635260959283
2000
0.10065931853641351
2100
0.10569228446323418
2200
0.11072525039005486
2300
0.11575821631687554
2400
0.12079118224369621
2500
0.12582414817051688
2600
0.13085711409733755
2700
0.13589008002415823
2800
0.1409230459509789
2900
0.14595601187779958
3000
0.15098897780462026
3100
0.15602194373144093
3200
0.1610549096582616
3300
0.16608787558508228
3400
0.17112084151190296
3500
0.17615380743872364
3600
0.1811867733655443
3700
0.186219739292365
3800
0.19125270521918566
3900
0.19628567114600634
4000
0.20131863707282702
4100

In [21]:
df['expandedText'] = outputColumn
df = df[['id', 'expandedText', 'label']]

In [25]:
df.to_json('expanded_data.json')